## Save and Restore TensorFlow Models

### First we will save a simple model

In [1]:
import tensorflow as tf

# archive path to save the data
save_file = './model.ckpt'

# two tensorflow's variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))

# class used for save and/or restore tensorflow's variables
saver = tf.train.Saver()

In [2]:
with tf.Session() as sess:
    # initialize al the variables
    sess.run(tf.global_variables_initializer())
    
    # print the weights and bias values
    print('Weights:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))
    
    # save the model
    saver.save(sess, save_file)

Weights:
[[-1.04737616 -0.41627699 -0.91335905]
 [ 1.50075734  0.14761274 -0.90812314]]
Bias:
[-1.5768038  -1.50132298 -0.5337891 ]


### Retore the variables

In [3]:
# remove the last model weights and bias
tf.reset_default_graph()

# two tensorflow variables: weights and biases
weights = tf.Variable(tf.truncated_normal([2,3]))
bias = tf.Variable(tf.truncated_normal([3]))

# class used for save and/or restore tensorflow's variables
saver = tf.train.Saver()

In [4]:
with tf.Session() as sess:
    # restore weights and biases
    saver.restore(sess, save_file)
    
    # show the weights and biases
    print('Weights:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

Weights:
[[-1.04737616 -0.41627699 -0.91335905]
 [ 1.50075734  0.14761274 -0.90812314]]
Bias:
[-1.5768038  -1.50132298 -0.5337891 ]


### Now saving a trained model

Preparing the model

In [5]:
# removing tensors last model
tf.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

# hyperparameters
learning_rate = 0.001
n_input = 784
n_classes = 10

# load mnist data
mnist = input_data.read_data_sets('.', one_hot = True)

# features and labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# weights and biases
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# logits - linear combination
logits = tf.add(tf.matmul(features, weights), bias)

# define the loss and the optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

# compute accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


Training the model

In [6]:
import math

save_file = './train_model.ckpt'
batch_size = 128
n_epochs = 100

saver = tf.train.Saver()

# launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # training cycle
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples/batch_size)
        
        # loop over all batches
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict = {features: batch_features, labels: batch_labels})
            
        # print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
            accuracy, 
            feed_dict = {
                features: mnist.validation.images,
                labels: mnist.validation.labels})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
            epoch, 
            valid_accuracy))
            
    # save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

Epoch 0   - Validation Accuracy: 0.13539999723434448
Epoch 10  - Validation Accuracy: 0.31360000371932983
Epoch 20  - Validation Accuracy: 0.42500001192092896
Epoch 30  - Validation Accuracy: 0.5012000203132629
Epoch 40  - Validation Accuracy: 0.5633999705314636
Epoch 50  - Validation Accuracy: 0.6078000068664551
Epoch 60  - Validation Accuracy: 0.647599995136261
Epoch 70  - Validation Accuracy: 0.6736000180244446
Epoch 80  - Validation Accuracy: 0.6949999928474426
Epoch 90  - Validation Accuracy: 0.7121999859809875
Trained Model Saved.


Loading the trained model

In [7]:
saver = tf.train.Saver()

# launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)
    
    test_accuracy = sess.run(
    accuracy, 
    feed_dict = {features: mnist.test.images, labels: mnist.test.labels})
    
print('Precisão de Teste: {}'.format(test_accuracy))

Precisão de Teste: 0.7290999889373779


### Launching the weights and biases in a new model

In [8]:
import tensorflow as tf

# remove weights and bias
tf.reset_default_graph()

save_file = './train_model.ckpt'

# two tensorflow variables: weights and biases
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()

# show the name of variables weight and bias
print('Weight Variable Saved: {}'.format(weights.name))
print('Bias Variable Saved: {}'.format(bias.name))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, save_file)
    
# remove last weights and bias
tf.reset_default_graph()

# two tensorflow variables: weights and bias
bias = tf.Variable(tf.truncated_normal([3]))
weights = tf.Variable(tf.truncated_normal([2, 3]))

save = tf.train.Saver()

# show the name of the variables weights and bias
print('Weight Variable used to load the data: {}'.format(weights.name))
print('Bias Variable used to load the data: {}'.format(bias.name))

with tf.Session() as sess:
    saver.restore(sess, save_file)

Weight Variable Saved: Variable:0
Bias Variable Saved: Variable_1:0
Weight Variable used to load the data: Variable_1:0
Bias Variable used to load the data: Variable:0


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [2,3] rhs shape= [3]
	 [[Node: save/Assign_1 = Assign[T=DT_FLOAT, _class=["loc:@Variable_1"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_1, save/RestoreV2_1)]]

Caused by op 'save/Assign_1', defined at:
  File "C:\Users\Marco\Anaconda3\envs\dlnd-tf-lab\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Marco\Anaconda3\envs\dlnd-tf-lab\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Marco\Anaconda3\envs\dlnd-tf-lab\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\Marco\Anaconda3\envs\dlnd-tf-lab\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Marco\Anaconda3\envs\dlnd-tf-lab\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.ins

In [9]:
import tensorflow as tf

tf.reset_default_graph()

save_file = './train_model.ckpt'

# two tensorflow variable: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]), name = 'weights_0')
bias = tf.Variable(tf.truncated_normal([3]), name = 'bias_0')

saver = tf.train.Saver()

# show the name of the variables weights and bias
print('Weights Variable Saved: {}'.format(weights.name))
print('Bias Variable Saved: {}'.format(bias.name))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, save_file)
    
# remove the last weights and biases values
tf.reset_default_graph()

# two tensorflow variable: weights and bias
bias = tf.Variable(tf.truncated_normal([3]), name = 'bias_0')
weights = tf.Variable(tf.truncated_normal([2, 3]), name = 'weights_0')

saver = tf.train.Saver()

# show the weights and bias variable name
print('Weights Variables used to load the data: {}'.format(weights.name))
print('Bias Variable used to load the data: {}'.format(bias.name))

with tf.Session() as sess:
    saver.restore(sess, save_file)
    
print('Weights and bias successfully loaded.')

Weights Variable Saved: weights_0:0
Bias Variable Saved: bias_0:0
Weights Variables used to load the data: weights_0:0
Bias Variable used to load the data: bias_0:0
Weights and bias successfully loaded.
